In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

import json

import os
import re

from colorama import Fore as fore, Back, Style

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.options.plotting.backend = "plotly"
np.set_printoptions(precision=4)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from collections import Counter
import matplotlib.pyplot as plt


df = pd.read_csv('Homework_2_data.csv', sep=';').query('year == 2023 & region == "Томск"').rename(
columns={"is_children": 'has_children', 'female': 'is_female', 'internet': 'used_internet', 
        'alcohol': 'use_alcohol', 'health': 'health_issues', 'smoke': 'smoking', 'regions': 'region'}).drop(columns='year')

for columns in 'weight', 'height', 'age', 'wage':
    df[columns] = df[columns].str.replace(',', '.').astype(float)

df.sort_values(by='wage', ascending=False, inplace=True)
df['high_educ'] = (df['educ'] == 2).astype(int)

FileNotFoundError: [Errno 2] No such file or directory: 'Homework_2_data.csv'

# Ex 1

In [164]:
formula

'wage ~ age + high_educ + is_female + smoking'

In [163]:
from statsmodels.formula.api import ols

needed_columns = ['age', 'high_educ', 'is_female', 'smoking']

formula = 'wage ~ ' + ' + '.join(needed_columns)

simple = ols(formula, df).fit()
print(simple.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.219
Model:                            OLS   Adj. R-squared:                  0.198
Method:                 Least Squares   F-statistic:                     10.43
Date:                Thu, 12 Dec 2024   Prob (F-statistic):           1.77e-07
Time:                        11:13:51   Log-Likelihood:                -1811.7
No. Observations:                 154   AIC:                             3633.
Df Residuals:                     149   BIC:                             3649.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   7.073e+04   1.18e+04      5.974      0.0

    Все коэффициенты оказались значимы на уровне значимости в 5% за исключением `smoking`. Его p-value составило 0.122, что незначимо на любом разумном уровне.

    p-value F-статистики, отвечающей за значимость модели в целом составило 1.77e-7, что значимо на любом разумном уровне значимости.
    
    R^2 составил 0.219 - следовательно - модель объяснила 21.9% процентов дисперсии целевой переменной
    
    Итоговая модель: Y ~ 70733.4 - 569.08 * age + 25461.57 * high_educ - 20226.09 * is_female + 8598.00 * smoking 

# Ex 2

In [162]:
formula

'wage_log ~ age + high_educ + is_female + smoking'

In [161]:
from statsmodels.formula.api import ols


df['wage_log'] = np.log(df['wage'])

needed_columns = ['age', 'high_educ', 'is_female', 'smoking']

formula = 'wage_log ~ ' + ' + '.join(needed_columns)

log_w = ols(formula, df).fit()
print(log_w.summary())

                            OLS Regression Results                            
Dep. Variable:               wage_log   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.236
Method:                 Least Squares   F-statistic:                     12.85
Date:                Thu, 12 Dec 2024   Prob (F-statistic):           5.21e-09
Time:                        11:12:52   Log-Likelihood:                -127.90
No. Observations:                 154   AIC:                             265.8
Df Residuals:                     149   BIC:                             281.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     11.0144      0.211     52.123      0.0

1. **Значимость коэффициентов**:  
   - Коэффициент для age: Значим на уровне значимости 5% c p-value = 0.022. Это говорит о том, что возраст статистически значимо влияет на логарифм заработной платы (wageLog).
   - Коэффициент для highEduc: Значим на уровне 5% с p-value -> 0.000. Высшее образование имеет сильное статистически значимое положительное влияние на wageLog.
   - Коэффициент для isFemale: Тоже значим с с p-value -> 0.000. Пол (isFemale) оказывает отрицательное влияние на wageLog, что может указывать на гендерный разрыв в заработной плате.
   - Коэффициент для smoking: **Не значим** на любом разумном уровне c p-value = 0.624. Таким образом, курение (smoking) не оказывает статистически значимого влияния на целевую переменную.


2. **Значимости модели в целом**:  
   - p-value F-статистики имеет значение 5.21e-09, что значительно ниже стандартных уровней значимости. Это говорит о том, что модель в целом статистически значима, то есть выбранный набор предикторов объясняет логарифм заработной платы лучше, чем константа


3. **Качество модели**:  
   - Коэффициент детерминации R^2 = 0.256: Это значит, что модель объясняет 25.6% дисперсии зависимой переменной wageLog. Это умеренный уровень объясняющей способности, указывающий на то, что часть вариации заработной платы связана с другими, не включенными в модель, факторами.
   - Adj. R^2 = 0.236: Учитывает число предикторов в модели. Больше нуля, поэтому объясняющая способность у модели действительно присутствует


4. **Итоговая модель**:  
   - Уравнение регрессии выглядит следующим образом:
   
   $$
   \text{wage log} = 11.0144 - 0.0090 \cdot \text{age} + 0.5002 \cdot \text{highEduc} - 0.4760 \cdot \text{isFemale} + 0.0485 \cdot \text{smoking}.
   $$

   **Интерпретация коэффициентов**:
   - Константа (Intercept = 11.0144): Средний уровень логарифма заработной платы, если все независимые переменные равны нулю.
   - age: С увеличением возраста на один год (при прочих равных) логарифм заработной платы уменьшается в среднем на 0.009.
   - higheduc: Получение высшего образования (в сравнении с отсутствием такового) в среднем увеличивает логарифм заработной платы на 0.5002.
   - isfemale: Пол "женщина" (в сравнении с "мужчина") снижает логарифм заработной платы в среднем на 0.4760.
   - smoking: Курение в среднем увеличивает логарифм заработной платы на 0.0485, но этот эффект незначим.
   
   **Интерпретация логарифма** 
   
   - Так как логарифм привращает мультипликативные соотношения в аддитивные, коэф при highEduc = 0.5002 в действительности можно интерпретировать, как то, что получение высшего образование увеличивает доход в e^0.5002 раз

# Ex 3

In [15]:
df['log_age'] = np.log(df['age'])

from statsmodels.formula.api import ols

needed_columns = ['log_age', 'high_educ', 'is_female', 'smoking']

formula = 'wage_log ~ ' + ' + '.join(needed_columns)

log_age_wage = ols(formula, df).fit()
print(log_age_wage.summary())

                            OLS Regression Results                            
Dep. Variable:               wage_log   R-squared:                       0.251
Model:                            OLS   Adj. R-squared:                  0.231
Method:                 Least Squares   F-statistic:                     12.51
Date:                Wed, 11 Dec 2024   Prob (F-statistic):           8.46e-09
Time:                        19:34:18   Log-Likelihood:                -128.42
No. Observations:                 154   AIC:                             266.8
Df Residuals:                     149   BIC:                             282.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     12.0246      0.700     17.180      0.0

1. **Значимость коэффициентов**: 

   - Коэффициент для LogAge: Значим на уровне значимости 5% с p-value = 0.039. Это говорит о том, что логарифм возраста (LogAge) статистически значимо влияет на логарифм заработной платы (WageLog).  
   - Коэффициент для HighEduc: Значим на уровне 1% с p-value = 0.000. Высшее образование (HighEduc) оказывает сильное положительное влияние на WageLog.  
   - Коэффициент для IsFemale: Тоже значим на уровне 1% с p-value = 0.000. Пол (IsFemale) оказывает отрицательное влияние на WageLog, что может указывать на гендерный разрыв в заработной плате.  
   - Коэффициент для Smoking: Не значим на любом разумном уровне (p-value = 0.586). Таким образом, курение (Smoking) не оказывает статистически значимого влияния на целевую переменную.  


2. **Значимость модели в целом**: 

   p-value F-статистики имеет значение 8.46e-09, что значительно ниже стандартных уровней значимости (например, 0.05). Это говорит о том, что модель в целом статистически значима, то есть выбранный набор предикторов объясняет логарифм заработной платы лучше, чем модель без предикторов.  


3. **Качество модели**:  

   - Коэффициент детерминации R-squared = 0.251: Это значит, что модель объясняет 25.1% дисперсии зависимой переменной (WageLog). Это умеренно низкий уровень объясняющей способности, указывающий на то, что часть вариации заработной платы связана с другими, не включенными в модель, факторами.  
   - Adj. R-squared = 0.231: Учитывает количество предикторов в модели и показывает сходный уровень объясняющей способности.  


4. **Итоговая модель**:  

   Уравнение регрессии выглядит следующим образом:  
   WageLog = 12.0246 - 0.3772 * LogAge + 0.5070 * HighEduc - 0.4785 * IsFemale + 0.0540 * Smoking  


   **Интерпретация коэффициентов**: 
   
   - Константа (Intercept = 12.0246): Средний уровень логарифма заработной платы, если все независимые переменные равны нулю. 
   - LogAge: С увеличением LogAge на одну единицу (при прочих равных) логарифм заработной платы уменьшается в среднем на 0.3772. Это указывает на обратную связь между возрастом (в логарифмической шкале) и заработной платой. Изменение логарифма по логарифму называется эластичностью и показывает соотношения процентных изменений регрессора и целевой переменной. Т.е. увеличение возраста на 1%, уменьшает зп на 0.3772%
   - HighEduc: Получение высшего образования (в сравнении с отсутствием такового) в среднем увеличивает логарифм заработной платы на 0.5070. Это сильный положительный эффект.  
   - IsFemale: Пол "женщина" (в сравнении с "мужчина") снижает логарифм заработной платы в среднем на 0.4785. Это указывает на наличие значительного гендерного разрыва в заработной плате.  
   - Smoking: Курение в среднем увеличивает логарифм заработной платы на 0.0540, но этот эффект статистически незначим.  


# Ex 4 (Надо доделать)

### Log(age)?

Поскольку в каждой регресси участвовало одинаковое число регрессоров, то наивно мы можем сравнить модели, сравнивая обычный R^2

Это было бы проще реализовать руками и проще объяснить "бизнесу"

$R^2_{linear}$ = 0.219

$R^2_{wageLog}$ = 0.256

$R^2_{wageAndAgeLog}$ = 0.251

Поэтому выбирая модель, мы точно предпочтём не логарифмировать возраст человека. 

Осталось выбрать какая модель лучше описывает взаимосвязь: с логарифмом зарплаты или просто с зарплатой. 

### Log or not to log (Wage)?

Сравним качество оценки взаимосвязи с помощью теста замбреки. Поскольку он является тайным знанием, доступным только избранным (студентам ВШЭ), то нам придётся писать его руками

In [319]:
from scipy.stats.mstats import gmean

g_mean = gmean(df['wage'])

df['wage_norm'] = df['wage'] / g_mean

from statsmodels.formula.api import ols

df['wage_norm_log'] = np.log(df['wage_norm'])

needed_columns = ['age', 'high_educ', 'is_female', 'smoking']

formula = 'wage_norm_log ~ ' + ' + '.join(needed_columns)
log_w_norm = ols(formula, df).fit()

formula = 'wage_norm ~ ' + ' + '.join(needed_columns)
simple_norm = ols(formula, df).fit()

rss_log = log_w_norm.centered_tss - log_w_norm.ess
rss_lin = simple_norm.centered_tss - simple_norm.ess

from scipy.stats import chi2

test_stat = len(df) / 2 * np.log(max(rss_log, rss_lin) / min(rss_log, rss_lin))

p_value = 1 - chi2.cdf(test_stat, df=1)

print(f'p-value({p_value:.6f})')

if p_value > 0.05:
    print('hypothesis that models are equal is not rejected')
elif rss_log < rss_lin:
    print('model with log_wage is better')
else:
    print('model with linear wage is better')

# Ex 5

### Test Chow

In [341]:
from statsmodels.formula.api import ols


# age - usual, log_wage
needed_columns = ['age', 'high_educ', 'smoking']
formula = 'wage_log ~ ' + ' + '.join(needed_columns)


log_wage = ols(formula, df).fit()
log_wage_fem = ols(formula, df.query('is_female == 1')).fit()
log_wage_male = ols(formula, df.query('is_female == 0')).fit()


rss = log_wage.centered_tss - log_wage.ess
rss_fem = log_wage_fem.centered_tss - log_wage_fem.ess
rss_male = log_wage_male.centered_tss - log_wage_male.ess

k, n = len(needed_columns) + 1, len(df)

numerator = (rss - rss_fem - rss_male) / k 
denomenator = (rss_fem + rss_male) / (n - 2 * k)

F = numerator / denomenator

from scipy.stats import f

p_value = 1 - f.cdf(F, k, n - 2 * k)

print(f'p-value is ({p_value:.3f})')

p-value is (0.000)


### Dummy variables test

In [342]:
for column in ['age', 'high_educ', 'smoking']:
    df[f'{column}_fem'] = df[column] * df['is_female']


needed_columns = ['age', 'high_educ', 'is_female', 'smoking']
formula = 'wage_log ~ ' + ' + '.join(needed_columns + [f'{column}_fem' for column in needed_columns if column != "is_female"])

dummy_reg = ols(formula, df).fit()

rss_dummy = dummy_reg.centered_tss - dummy_reg.ess

numerator = (rss - rss_dummy) / k 
denomenator = (rss_dummy) / (n - 2 * k)


F = numerator / denomenator

from scipy.stats import f

p_value = 1 - f.cdf(F, k, n - 2 * k)

print(f'p-value is ({p_value:.3f})')

p-value is (0.000)


p-value нулевой по обоим тестам (imagine my shock), поэтому действительно присутствует структурный сдвиг в уравнениях регрессий зарплаты мужчин и женщин 

# Ex 6

### Стьюдентизированные остатки

In [98]:
from statsmodels.stats.outliers_influence import OLSInfluence

df['outliers_student'] = (np.abs(OLSInfluence(simple).resid_studentized_internal) > 2).astype(int)

if df['outliers_student'].sum() > 0:
    print('Есть выбросы согласно методу стьюдентизированных остатков')
else:
    print('Выбросов не обнаружено')

Есть выбросы согласно методу стьюдентизированных остатков


### DFBETA estimation

In [118]:
from statsmodels.stats.outliers_influence import OLSInfluence


params = log_w.conf_int().reset_index()['index'].tolist()

changes = OLSInfluence(log_w).dfbetas

for index, column_name in enumerate(params):
    fig = px.histogram(changes[:, index], title=f'histogram of DFBETAS with feature({column_name})',
                      labels={'x': 'df betas'})
    fig.show()

Воспользуюсь таким способом аггрегации: возьму среднее модулей изменений параметров по всем признакам

In [126]:
aggregated = np.mean(np.abs(changes), axis=1)

In [127]:
fig = px.histogram(aggregated, title=f'histogram of aggregated DFBETAS',
                  labels={'x': 'df betas'})
fig.show()

Справедливо можно заметить, что 3 наблюдения "больше" 0.21 отделены от всех остальных. Их мы и возьмём

In [133]:
df['dfbetas_outlier'] = (aggregated > 0.21).astype(int)

Получается, что также согласно dfbeta критерию, в выборке есть выбросы. 

# Ex 7 

### VIF

Для начала протестируем наличие мультиколлинеарности с помощью раздувателя дисперсии (говорим по-русски)

In [135]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [201]:
df

,region,educ,age,is_female,industry,work_hours,wage,foreign_language,used_internet,use_alcohol,has_children,health_issues,weight,height,smoking,high_educ,wage_log,log_age,wage_fem,age_fem,high_educ_fem,smoking_fem,too_good_to_be_true,outliers_student,dfbetas,dfbetas_outlier,intercept
2722,Томск,2,33.0,1,"ТОРГОВЛЯ, БЫТОВОЕ ОБСЛУЖИВАНИЕ",8,280000.0,1,1,1,1,0,76.0,168.0,1,1,12.542545,3.496508,1,33.0,1,1,1,1,0.269163,1,1
3815,Томск,2,39.0,0,ДРУГАЯ ОТРАСЛЬ ТЯЖЕЛОЙ ПРОМЫШЛЕННОСТИ,12,200000.0,1,1,1,1,0,93.0,178.0,0,1,12.206073,3.663562,0,0.0,0,0,1,1,0.154746,0,1
3690,Томск,2,40.5,0,ОРГАНЫ УПРАВЛЕНИЯ,12,173000.0,1,1,1,1,0,83.0,183.0,1,1,12.061047,3.701302,0,0.0,0,0,1,1,0.117115,0,1
2454,Томск,2,39.5,1,"ДЕРЕВООБРАБАТЫВАЮЩАЯ ПРОМЫШЛЕННОСТЬ, ЛЕСНОЕ ХОЗЯЙСТВО",8,120000.0,0,1,1,1,0,58.0,164.0,1,1,11.695247,3.676301,1,39.5,1,1,0,0,0.123385,0,1
4260,Томск,1,39.0,0,"ЛЕГКАЯ, ПИЩЕВАЯ ПРОМЫШЛЕННОСТЬ",12,120000.0,0,1,1,1,1,90.0,174.0,0,0,11.695247,3.663562,0,0.0,0,0,1,1,0.182995,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,Томск,0,67.0,1,"ТОРГОВЛЯ, БЫТОВОЕ ОБСЛУЖИВАНИЕ",2,10000.0,0,1,0,1,1,115.0,173.5,0,0,9.210340,4.204693,0,67.0,0,0,0,0,0.113071,0,1
1994,Томск,0,63.0,1,"ТОРГОВЛЯ, БЫТОВОЕ ОБСЛУЖИВАНИЕ",7,10000.0,0,1,1,1,1,89.0,156.0,1,0,9.210340,4.143135,1,63.0,0,1,0,0,0.143255,0,1
1146,Томск,2,45.5,1,"РЕКЛАМА, МАРКЕТИНГ",4,10000.0,1,1,1,1,0,60.0,160.0,0,1,9.210340,3.817712,0,45.5,1,0,0,0,0.095112,0,1
4542,Томск,1,50.5,1,"ТОРГОВЛЯ, БЫТОВОЕ ОБСЛУЖИВАНИЕ",8,6500.0,0,1,0,1,1,83.0,160.0,1,0,8.779557,3.921973,1,50.5,0,1,0,0,0.159961,0,1


In [198]:
needed_columns = ['age', 'high_educ', 'is_female', 'smoking']
df['intercept'] = 1

In [202]:
for index, column in enumerate(needed_columns):
    VIF = variance_inflation_factor(df[needed_columns + ['intercept']].values, index)
    
    print(f'for column({column}) VIF({VIF:.4f})')

for column(age) VIF(1.0315)
for column(high_educ) VIF(1.1462)
for column(is_female) VIF(1.0913)
for column(smoking) VIF(1.1278)


Видим, что каждый отдельный признак не очень хорошо предсказывается остальными, но они вместо хорошо складываются в константу. Поэтому мультиколлинеарность присутствует 

### Можно посмотреть на распределения сингулярных чисел (на число обусловленности матрицы), чтобы обнаружить сильную зависимость между колонками.

Для начала давайте стандартизируем фичи: вычтем из каждой среднее и поделим на стандартное отклонение

In [210]:
to_test = df[needed_columns].copy()

for column in needed_columns:
    to_test[column] = (to_test[column] - to_test[column].mean()) / to_test[column].std()

In [211]:
np.linalg.svd(to_test)[1]

array([15.2267, 12.4297, 11.2009, 10.0096])

In [212]:
singular_values = np.linalg.svd(to_test)[1]

condition_number = singular_values[0] / singular_values[-1]

print(f'condition number({condition_number:.4f})')

condition number(1.5212)


Относительно небольшое число обусловленности, сингулярные числа не говорят о том что в данных есть большая мультиколлинеарность (они не могут учитывать наличие intercept в данных так как его не отнормировать).

# Ex 8

Минусы - чуть тяжелее интерпретировать значения линейных признаков

При этом плюсы - дисперсии бет становятся ниже (1), мы можем выкинуть шумовые признаки (2),
избавиться от мультиколлинеарности - признаки становятся "чище" (3), снижается вычислительная сложность (4), данные можно взиуализировать (5d -> 3d или 3d).

PCA всё же предназначен чтобы работать с непрерывными данными, а у меня все колонки кроме age - бинарные. Так же у нас нет проблемы с мультиколлинеарностью или вычислительными мощностями. Поэтому я бы не стал использовать PCA.

Применю обычное SVD - так мне проще)

In [225]:
U, S, V_T = np.linalg.svd(to_test, full_matrices=False)

US = XV

In [228]:
to_test

,age,high_educ,is_female,smoking
2722,-1.203195,0.983886,0.863209,1.230728
3815,-0.698885,0.983886,-1.150945,-0.807251
3690,-0.572807,0.983886,-1.150945,1.230728
2454,-0.656859,0.983886,0.863209,1.230728
4260,-0.698885,-1.009778,-1.150945,-0.807251
...,...,...,...,...
227,1.654564,-1.009778,0.863209,-0.807251
1994,1.318357,-1.009778,0.863209,1.230728
1146,-0.152548,0.983886,0.863209,-0.807251
4542,0.267710,-1.009778,0.863209,1.230728


In [229]:
V_T.T

array([[ 0.2465, -0.9082, -0.019 ,  0.3376],
       [ 0.5717,  0.4167, -0.1182,  0.6969],
       [ 0.5305,  0.0127,  0.7893, -0.3089],
       [-0.5753,  0.0365,  0.6023,  0.5523]])

**Получили такую матрицу**

Первая главная компонента за мудрых женщин, которые отказались от курения

Вторая за молодых людей с высшим образованием

Третья за больше про курящих женщин

Последняя про взрослых, образованных и курящих мужчин

# Ex 9

In [239]:
px.scatter(x=log_w.resid, y=log_w.fittedvalues,
          title='The "residuals–forecasts" chart',
          labels={'x': 'residuals', 'y':'prediction'})

Выглядит как хорошее облако, я полагаю, что гомоскедостичность соблюдена)

# Ex 10

### Есть ли гетероскедастичность?

In [309]:
from statsmodels.stats.diagnostic import het_white

to_test = df[needed_columns].copy()


import statsmodels.api as sm

X_with_const = sm.add_constant(to_test)


lm_stat, lm_pvalue, f_stat, f_pvalue = het_white(log_w.resid ** 2, X_with_const)


if f_pvalue < 0.05:
    print("Гетероскедастичность на месте")
else:
    print("Гомоскедастичность есть")

Гетероскедастичность на месте


### Решаем

In [258]:
robust_model = log_w.get_robustcov_results(cov_type="HC0")

print(robust_model.summary())

                            OLS Regression Results                            
Dep. Variable:               wage_log   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.236
Method:                 Least Squares   F-statistic:                     15.29
Date:                Thu, 12 Dec 2024   Prob (F-statistic):           1.68e-10
Time:                        21:59:38   Log-Likelihood:                -127.90
No. Observations:                 154   AIC:                             265.8
Df Residuals:                     149   BIC:                             281.0
Df Model:                           4                                         
Covariance Type:                  HC0                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     11.0144      0.205     53.658      0.0

# Ex 11

In [308]:
from statsmodels.formula.api import ols

df['wage_log'] = np.log(df['wage'])
df['intercept'] = 1

needed_columns = ['age', 'high_educ', 'is_female', 'smoking', 'intercept']

log_w = sm.OLS(df['wage_log'].to_numpy(), 
            df[needed_columns].to_numpy()).fit()


from statsmodels.stats.diagnostic import linear_reset

reset = linear_reset(res=log_w, power=2, test_type="fitted", use_f=True)

p_value = reset.pvalue

print(f'p-value({p_value:.6f})')

if reset.pvalue < 0.05:
    print("есть нелинейность по параметрам")
else:
    print("модель в этом плане специфицирована правильно")

p-value(0.200704)
модель в этом плане специфицирована правильно


# Ex 12

In [296]:
from statsmodels.formula.api import ols


df['wage_log'] = np.log(df['wage'])

needed_columns = ['age', 'high_educ', 'is_female', 'smoking']

formula = 'wage_log ~ ' + ' + '.join(needed_columns)

log_w = ols(formula, df).fit()
print(log_w.summary())

                            OLS Regression Results                            
Dep. Variable:               wage_log   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.236
Method:                 Least Squares   F-statistic:                     12.85
Date:                Fri, 13 Dec 2024   Prob (F-statistic):           5.21e-09
Time:                        09:32:03   Log-Likelihood:                -127.90
No. Observations:                 154   AIC:                             265.8
Df Residuals:                     149   BIC:                             281.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     11.0144      0.211     52.123      0.0

Smoking оказался незначимым. Поэтому уберём его из оценки

In [297]:
needed_columns = ['age', 'high_educ', 'is_female']

formula = 'wage_log ~ ' + ' + '.join(needed_columns)

simple = ols(formula, df).fit()
print(simple.summary())

                            OLS Regression Results                            
Dep. Variable:               wage_log   R-squared:                       0.255
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     17.13
Date:                Fri, 13 Dec 2024   Prob (F-statistic):           1.28e-09
Time:                        09:33:37   Log-Likelihood:                -128.03
No. Observations:                 154   AIC:                             264.1
Df Residuals:                     150   BIC:                             276.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     11.0550      0.194     56.981      0.0

Нельзя сравнивать обычный $R^2$ потому что при добавлении любого признака (даже незначимого), то $R^2$ не уменьшится. Поэтому следует сравнивать другой признак, например, "скорректированный" $R^2$. Для модели с удалённым smoking он составил 0.24, с ним - 0.236, поэтому можно сказать, что короткая модель "лучше".

# Ex 13

Интуитивно влияение возраста на доход нелинейное. На самом деле молодые получают мало из-за отсутствия высшего образовани и/или маленького опыта, люди предпенсионного и пенсионного возраста получают меньше из-за снижения работоспособности и приближения пенсии (работодатель понимает, что работник вскоре может уйти и ему придётся найти замену, поэтому предпочитает не нанимать таких сотрудников.

Тогда в корректной функциональной форме зависимость будет не только от возраста, но и от квадрата возраста.

Сформулируем гипотезу так: 

wage_log ~ age + high_educ + is_female + is_smoking + age^2

H0: $ \beta_5 = 0$

H1: $ \beta_5 \ne 0$

Проверить такую гипотезу можно обычным т-тестом

### T-test

In [300]:
df['age_squared'] = df['age'] ** 2

In [301]:
needed_columns = ['age', 'high_educ', 'is_female', 'smoking', 'age_squared']

formula = 'wage_log ~ ' + ' + '.join(needed_columns)

simple = ols(formula, df).fit()
print(simple.summary())

                            OLS Regression Results                            
Dep. Variable:               wage_log   R-squared:                       0.264
Model:                            OLS   Adj. R-squared:                  0.239
Method:                 Least Squares   F-statistic:                     10.63
Date:                Fri, 13 Dec 2024   Prob (F-statistic):           9.77e-09
Time:                        09:41:04   Log-Likelihood:                -127.10
No. Observations:                 154   AIC:                             266.2
Df Residuals:                     148   BIC:                             284.4
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      10.2154      0.674     15.149      

### Давайте проверим ещё с помощью сравнения p-value тестов Рамсея по аналогии со слайдами лекции

In [307]:
from statsmodels.formula.api import ols

df['intercept'] = 1

needed_columns = ['age', 'high_educ', 'is_female', 'smoking', 'intercept', 'age_squared']

log_w = sm.OLS(df['wage_log'].to_numpy(), 
            df[needed_columns].to_numpy()).fit()


from statsmodels.stats.diagnostic import linear_reset

reset = linear_reset(res=log_w, power=2, test_type="fitted", use_f=True)

p_value = reset.pvalue

print(f'{p_value:.6f}')

0.161927


p-value > 0.05 -> модель правильно специфицирована

но p-value ниже, чем без age^2. Хз что это означать может

# Ex 14

Мы выяснили, что у нас нет разной функциональной зависимости от остальных параметров для мужчин и для женщин, поэтому будем использовать обычную модель

Мы будем использовать такую функциональную зависимость (она дала лучший скор - смотри пункт 4):

wage_log ~ age + high_educ + is_female (модель без smoking оказалась лучше)

В итоге мы выяснили, что в нашем датасете нет проблемы с мультиколлинеарности, поэтому не будем применять PCA (или другие методы борьбы с мультиколлинеарностью), но у нас нарушается предположение о гомоскедастичности, поэтому мы будем использовать робастные стандартные ошибки


In [318]:
needed_columns = ['age', 'high_educ', 'is_female']

formula = 'wage_log ~ ' + ' + '.join(needed_columns)

final = ols(formula, df).fit()

final.get_robustcov_results(cov_type="HC0")
print(final.summary())

                            OLS Regression Results                            
Dep. Variable:               wage_log   R-squared:                       0.255
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     17.13
Date:                Fri, 13 Dec 2024   Prob (F-statistic):           1.28e-09
Time:                        09:59:21   Log-Likelihood:                -128.03
No. Observations:                 154   AIC:                             264.1
Df Residuals:                     150   BIC:                             276.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     11.0550      0.194     56.981      0.0